## Algoritmo genético que maximiza a função
### $f(x_{1}, x_{2}) =  21.5 + x_{1}sen(4 \pi x_{1}) + x_{2}sen(20 \pi x_{2})$

* variável $x_{1}$ definida no intervalo real [ –3.1; 12.1 ]
* variável $x_{2}$ definida no intervalo real [ 4.1; 5.8 ]


<!--
#teste forca bruta
max_x = max_y = maximo = 0     

for x in np.arange(-3.1, 12.1, 0.001):
    for y in np.arange(4.1, 5.8, 0.001):
        v = 21.5 + x * sin(4*pi*x) + y * sin(20*pi*y)
        if v > maximo:
            maximo = v
            max_x = x
            max_y = y
            
display(maximo)
display(max_x)
display(max_y)

'''
max: 38.85008205991546
x: 11.62599999999838
y: 5.725000000000542
'''
-->

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd

import random
from math import sin, pi
from time import sleep

from ipywidgets import *
import ipywidgets as widgets

from IPython.display import display, HTML

class Individuo():   
    
    def __init__(self, qtd_cromossomos, geracao=1, **kwargs):
        self.qtd_cromossomos = qtd_cromossomos
        self.cromossomo = np.array([random.randint(0, 1) for i in range(qtd_cromossomos)])
        self.geracao = geracao
        self.aptidao = 0
        self.x1 = 0
        self.x2 = 0
        self.x1_sup = 12.1
        self.x1_inf = -3.1
        self.x2_sup = 5.8
        self.x2_inf = 4.1
                        
    def __repr__(self): # formata representacao do objeto quanto utiliza print/display
        return "{} - Apt:{:8.3f} - x1:{:8.3f} - x2:{:8.3f} - Geracao: {}".format(self.cromossomo, self.aptidao, 
                                                                  self.x1, self.x2,self.geracao)
    
    def avaliacao(self):
        r1 = self.cromossomo[0:int(self.qtd_cromossomos/2)]
        r2 = self.cromossomo[int(self.qtd_cromossomos/2)::]
        r1 = int(''.join([str(i) for i in r1]), 2) #valor decimal da primeira metade
        r2 = int(''.join([str(i) for i in r2]), 2) #valor decimal da segunda metade
        x1 = self.x1_inf + ((self.x1_sup - self.x1_inf)/(2**(self.qtd_cromossomos/2) - 1) * r1)
        x2 = self.x2_inf + ((self.x2_sup - self.x2_inf)/(2**(self.qtd_cromossomos/2) - 1) * r2)
        self.aptidao = 21.5 + x1 * sin(4*pi*x1) + x2 * sin(20*pi*x2)
        self.x1 = x1
        self.x2 = x2
        
    def cruzamento(self, outro_individuo, qtd_pontos):
        ponto = random.sample(range(1, self.qtd_cromossomos), qtd_pontos)
        ponto.sort()
        
        if qtd_pontos == 1:
            filho1_cromossomo = np.concatenate((self.cromossomo[0:ponto[0]], outro_individuo.cromossomo[ponto[0]::]))
            filho2_cromossomo = np.concatenate((outro_individuo.cromossomo[0:ponto[0]], self.cromossomo[ponto[0]::]))
        elif qtd_pontos == 2:
            filho1_cromossomo = np.concatenate((self.cromossomo[0:ponto[0]], outro_individuo.cromossomo[ponto[0]:ponto[1]],
                self.cromossomo[ponto[1]::]))
            filho2_cromossomo = np.concatenate((outro_individuo.cromossomo[0:ponto[0]], self.cromossomo[ponto[0]:ponto[1]],
                outro_individuo.cromossomo[ponto[1]::]))
        
        filhos = [Individuo(qtd_cromossomos=self.qtd_cromossomos, geracao=self.geracao+1),
                 Individuo(qtd_cromossomos=self.qtd_cromossomos, geracao=self.geracao+1)]
        filhos[0].cromossomo = filho1_cromossomo
        filhos[1].cromossomo = filho2_cromossomo
        
        return filhos
    
    def mutacao(self, taxa_mutacao):        
        for i in range(len(self.cromossomo)):
            if random.random() < taxa_mutacao:     
                self.cromossomo[i] = 0 if self.cromossomo[i] == 1 else 1
        return self
    

class AlgoritmoGenetico():
    
    def __init__(self, tam_populacao, qtd_cromossomos, taxa_cruzamento, taxa_mutacao,
                qtd_geracoes, selecao, elitismo, pontos, tam_torneio=0, **kwargs):
        self.tam_populacao = tam_populacao
        self.populacao = np.array([])
        self.qtd_cromossomos = qtd_cromossomos
        self.taxa_cruzamento = taxa_cruzamento
        self.taxa_mutacao = taxa_mutacao
        self.qtd_geracoes = qtd_geracoes
        self.geracao = 0
        self.selecao = selecao
        self.tam_torneio = tam_torneio
        self.elitismo = elitismo
        self.pontos = pontos
        self.lista_solucao = []
        self.melhor_solucao = 0
    
    def inicializa_populacao(self):   
        lista = []
        for i in range(self.tam_populacao):
            lista.append(Individuo(qtd_cromossomos=self.qtd_cromossomos))
        self.populacao = np.array(lista)
        self.melhor_solucao = self.populacao[0]    
        
    def ordena_populacao(self):
        self.populacao = sorted(self.populacao,
                               key =  lambda individuo: individuo.aptidao,
                               reverse = True)
        self.populacao = np.array(self.populacao)
        
    def calcula_aptidao(self):
        for ind in self.populacao:
            ind.avaliacao()
    
    def melhor_individuo(self, individuo):
        if individuo.aptidao > self.melhor_solucao.aptidao:
            self.melhor_solucao = individuo
    
    def soma_aptidoes(self):
        soma = 0
        for i in self.populacao:
            soma += i.aptidao
        return soma
            
    def seleciona_pais(self, soma_aptidoes=0):
        '''
        retorna um array com o par de pais com base no tipo de selecao
        '''
        if self.selecao == 'roleta':
            r_pais = [random.random() * soma_aptidoes]
            while True:
                r = random.random() * soma_aptidoes
                if r not in r_pais:
                    r_pais.append(r)
                    break
                    
            pais = []
            for r in r_pais:
                pai = -1
                i = 0
                soma = 0
                while i < len(self.populacao) and soma < r:
                    soma += self.populacao[i].aptidao
                    pai += 1
                    i += 1
                pais.append(self.populacao[pai])
            return pais
        
        elif self.selecao == 'torneio':
            indices_torneio = random.sample(range(0, self.tam_populacao), self.tam_torneio)
            torneio = self.populacao[indices_torneio]
            torneio = sorted(torneio, key=lambda i: i.aptidao, reverse=True)
            return [torneio[0], torneio[1]]
        
    
    def seleciona_elitismo(self):
        '''
        retorna lista contendo os individuos elitistas
        '''
        self.ordena_populacao()
        return self.populacao[0:self.elitismo]
    
    

########## entradas dos parametros ##########

print('\n\nTamanho do cromossomo:')
def f(cromossomo=10):
    return cromossomo
cromossomo = interactive(f, cromossomo=(6, 20, 2))
display(cromossomo)

print('\n\nTamanho da população:')
def f(populacao=50):
    return populacao
populacao = interactive(f, populacao=(10, 100, 10))
display(populacao)

print('\n\nProbabilidade de cruzamento:')
def f(cruzamento=95.0):
    return cruzamento
cruzamento = interactive(f, cruzamento=(5.0, 95.0, 0.5))
display(cruzamento)

print('\n\nProbabilidade de mutação:')
def f(mutacao=0.1):
    return mutacao
mutacao = interactive(f, mutacao=(1.0, 10.0, 0.5))
display(mutacao)

print('\n\nQuantidade de gerações:')
def f(geracoes=30):
    return geracoes
geracoes = interactive(f, geracoes=(10, 100))
display(geracoes)

print('\n\nMétodo de seleção:')
def f(selecao):
    return selecao
selecao = interactive(f, selecao=['Roleta', 'Torneio'])
display(selecao)

print("""\n\nTamanho do Torneio:\n *ignorar caso o 'Método de seleção' selecionado seja diferente de 'Torneio'*""")
def f(tam_torneio=10):
    return tam_torneio
tam_torneio = interactive(f, tam_torneio=(2, 50, 2))
display(tam_torneio)

print('\n\nTamanho do elitismo:')
def f(elitismo=1):
    return elitismo
elitismo = interactive(f, elitismo=(0, 2))
display(elitismo)

print('\n\nQuantidade de pontos nos cruzamentos:')
def f(pontos):
    return pontos
pontos = interactive(f, pontos=(1, 2))
display(pontos)


########## processamento ##########
def processar():
    
    alg = AlgoritmoGenetico(
        tam_populacao = populacao.result, 
        qtd_cromossomos = cromossomo.result,
        taxa_cruzamento = cruzamento.result/100.0,
        taxa_mutacao = mutacao.result/100.0,
        qtd_geracoes = geracoes.result,
        selecao = selecao.result.lower(),
        elitismo = elitismo.result,
        pontos = pontos.result,
        tam_torneio = tam_torneio.result)
    
    ## inicio ##
    
    alg.inicializa_populacao()
    
    x1 = np.arange(-3.1, 12.1, 0.01) #### quanto menor o intervalo,mais preciso o grafico mas mais lento fica o processamento
    x2 = np.arange(4.1, 5.8, 0.01)
    X1, X2 = np.meshgrid(x1, x2)
    Z = 21.5 + X1 * np.sin(4*pi*X1) + X2 * np.sin(20*pi*X2)
    melhores_x1 = []
    melhores_x2 = []
    melhores_apt = []
    
    for g in range(0, alg.qtd_geracoes):
        #print("Geracao {}".format(g+1))
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        
        
        nova_popu = []
        soma_apt = alg.soma_aptidoes() if alg.selecao == 'roleta' else 0
        for c in range(0, alg.tam_populacao, 2):
            pais = alg.seleciona_pais(soma_apt)
            if random.random() <= alg.taxa_cruzamento:
                filhos = pais[0].cruzamento(pais[1], alg.pontos)
                for filho in filhos:
                    nova_popu.append(filho.mutacao(alg.taxa_mutacao))
            else:
                for pai in pais:
                    nova_popu.append(pai)
        
        elitistas = alg.seleciona_elitismo()
        indices = random.sample(range(0, alg.tam_populacao), len(elitistas))
        for idx in range(0, len(elitistas)):
            alg.populacao[indices[idx]] = elitistas[idx]
            
        alg.populacao = np.array(nova_popu)
        
        alg.calcula_aptidao()
        alg.ordena_populacao()
        alg.lista_solucao.append(alg.populacao[0])
        alg.melhor_individuo(alg.populacao[0])
        melhores_x1.append(alg.melhor_solucao.x1)
        melhores_x2.append(alg.melhor_solucao.x2)
        melhores_apt.append(alg.melhor_solucao.aptidao)
        
        
    
    ########## graficos ##########
    fig = plt.figure(figsize=plt.figaspect(0.2), dpi=200)

    # grafico da superficie
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax.plot_surface(X1, X2, Z, cmap='inferno', alpha=0.5)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    fig.colorbar(surf, shrink=0.5, aspect=5)

    ax.scatter(melhores_x1, melhores_x2, melhores_apt)

    # grafico do contorno
    ax = fig.add_subplot(1, 2, 2)
    contorno = ax.contourf(X1, X2, Z, locator=LogLocator(), cmap='inferno')

    ax.scatter(melhores_x1, melhores_x2, melhores_apt)

    plt.show()
    ########## /graficos ##########

    ## fim ##
    #display(alg.lista_solucao)
    
    ########## graficos ##########
    fig = plt.figure(figsize=plt.figaspect(0.2), dpi=200)

    # grafico da superficie
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax.plot_surface(X1, X2, Z, cmap='inferno', alpha=0.5)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    fig.colorbar(surf, shrink=0.5, aspect=5)

    ax.scatter(alg.melhor_solucao.x1, alg.melhor_solucao.x2, alg.melhor_solucao.aptidao,
              label="{:8.3f}".format(alg.melhor_solucao.aptidao))
    ax.legend()

    # grafico do contorno
    ax = fig.add_subplot(1, 2, 2)
    contorno = ax.contourf(X1, X2, Z, locator=LogLocator(), cmap='inferno')

    ax.scatter(alg.melhor_solucao.x1, alg.melhor_solucao.x2, alg.melhor_solucao.aptidao,
              label="{:8.3f}".format(alg.melhor_solucao.aptidao))
    ax.legend()

    plt.show()
    ########## /graficos ##########
    string = "<h4><strong>Melhor solução:</strong> {:8.3f}</h4>" \
            "<h4><strong>X1:</strong> {:8.3f}</h4>" \
            "<h4><strong>X2:</strong> {:8.3f}</h4>" \
            "<h4><strong>Geração:</strong> {}</h4>".format(
        alg.melhor_solucao.aptidao, alg.melhor_solucao.x1, alg.melhor_solucao.x2,
        alg.melhor_solucao.geracao)
    display(HTML(string))
    
    
widgets.interact_manual.opts['manual_name'] = 'Processar algoritmo' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Tamanho do cromossomo:


interactive(children=(IntSlider(value=10, description='cromossomo', max=20, min=6, step=2), Output()), _dom_cl…



Tamanho da população:


interactive(children=(IntSlider(value=50, description='populacao', min=10, step=10), Output()), _dom_classes=(…



Probabilidade de cruzamento:


...



Tamanho do elitismo:


interactive(children=(IntSlider(value=1, description='elitismo', max=2), Output()), _dom_classes=('widget-inte…



Quantidade de pontos nos cruzamentos:


interactive(children=(IntSlider(value=1, description='pontos', max=2, min=1), Output()), _dom_classes=('widget…

interactive(children=(Button(description='Processar algoritmo', style=ButtonStyle()), Output()), _dom_classes=…